1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma langchain-pinecone  pinecone pinecone 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 1. 문서 내용 읽기고 분할 #################################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 백터 데이터베이스에 저장 ####################################
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정  
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 3. Pinecone vector DB
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-index',
)

## 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ######################
# 4.1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 4.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###################
## 5.1. 프롬프트 작성  
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question : {query}
'''

## 5.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 5.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 5.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기 피해자에게 제공되는 금융지원 제도는 피해자들이 경제적 어려움을 극복하고 재정적 안정을 되찾을 수 있도록 도와주기 위한 다양한 지원책을 포함합니다. 전세사기 피해자 금융지원에는 다음과 같은 내용이 포함될 수 있습니다:\n\n1. **긴급 생활자금 대출**: 전세사기 피해로 인해 갑작스러운 주거 불안을 겪는 피해자들에게 긴급 생활자금을 저리로 대출해주는 제도입니다. 주거 안정이 회복될 때까지 임시로 필요할 수 있는 생활비를 지원합니다.\n\n2. **전세금 반환 보증**: 전세보증금을 돌려받지 못한 피해자를 위한 보증 상품으로, 일정 기간 내에 전세금을 돌려받지 못하면 보증기관이 대신 지급하는 형태입니다.\n\n3. **주택 마련 자금 지원**: 피해자들이 새로운 주거지를 구하는 데 필요한 초기 자금(전세금 또는 보증금)을 낮은 금리로 대출해주는 금융상품을 제공할 수 있습니다.\n\n4. **채무 조정 및 탕감 프로그램**: 전세사기로 인해 생긴 채무 부담을 덜기 위한 조정 및 부분 탕감 프로그램이 제공될 수 있습니다. 이를 통해 채무 상환 기간을 연장하거나 원리금을 감면받을 수 있는 기회를 제공합니다.\n\n5. **상담 및 법률 지원**: 금융 지원과 더불어 피해자에게 심리적, 법률적 상담을 제공하여 피해자들이 올바른 법적 조처를 할 수 있도록 지원합니다.\n\n전세사기 피해에 대한 금융지원 제도는 나라와 해당 지역의 정책에 따라 다를 수 있으며, 이러한 지원을 받기 위해 관련 기관이나 단체에 문의하여 최신 정보를 확인하는 것이 좋습니다. 이를 통해 피해자가 최대한 빨리 재정적, 심리적 안정을 찾을 수 있도록 도울 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 428, 'prompt_tokens': 59, 'total_tokens': 487, 'completion_tokens_det